## Initial preparations

In [134]:
device = 'cuda' if torch.cuda else 'cpu'
device

'cuda'

## Dataset: Tiny Shakespeare

### Download

In [3]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [1]:
with open("input.txt", 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
print(len(text))

1115394


In [3]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



### Building vocabular

In [109]:
vocab = sorted(list(set(text)))
vocab_size = len(vocab)
print("".join(vocab))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


### Tokenizing

Revise tokenisers:

- SentencePiece powered by Google
- TikToken

In [110]:
stoi = {ch:i for i,ch in enumerate(vocab)}
itos = {i:ch for i,ch in enumerate(vocab)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda L: [itos[i] for i in L]

print(encode("hello, world!"))
print(''.join(decode(encode("hello, world!"))))

[46, 43, 50, 50, 53, 6, 1, 61, 53, 56, 50, 42, 2]
hello, world!


In [111]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

### Train/Validation split

In [112]:
train_size = int(0.9*len(data))
train_X    = data[:train_size]
val_X      = data[train_size:]
print(train_size)

1003854


### Setup block_size - maximum context length

- context length - count of tokens in a continious line prior target token

In [113]:
block_size = 8
train_X[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [114]:
example_x = train_X[:block_size]
example_y = train_X[1:block_size+1]
for t in range(block_size):
    context = example_x[:t+1]
    target = example_y[t]
    print(f"when input is {context} the target is {target}")

when input is tensor([18]) the target is 47
when input is tensor([18, 47]) the target is 56
when input is tensor([18, 47, 56]) the target is 57
when input is tensor([18, 47, 56, 57]) the target is 58
when input is tensor([18, 47, 56, 57, 58]) the target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [135]:
torch.manual_seed(1337)
batch_size = 4

def get_batch(split):
    data = train_X if split == 'train' else val_X
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x  = torch.stack([data[i:i+block_size] for i in ix])
    y  = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


In [137]:
xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)
print('----')
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[ 6,  0, 21, 44,  1, 61, 43,  1],
        [58, 52, 43, 57, 57,  2,  1, 57],
        [ 1, 59, 52, 39, 41, 46, 47, 52],
        [43, 42, 50, 39, 56,  6,  1, 50]], device='cuda:0')
targets:
torch.Size([4, 8])
tensor([[ 0, 21, 44,  1, 61, 43,  1, 61],
        [52, 43, 57, 57,  2,  1, 57, 43],
        [59, 52, 39, 41, 46, 47, 52, 45],
        [42, 50, 39, 56,  6,  1, 50, 43]], device='cuda:0')
----
when input is [6] the target: 0
when input is [6, 0] the target: 21
when input is [6, 0, 21] the target: 44
when input is [6, 0, 21, 44] the target: 1
when input is [6, 0, 21, 44, 1] the target: 61
when input is [6, 0, 21, 44, 1, 61] the target: 43
when input is [6, 0, 21, 44, 1, 61, 43] the target: 1
when input is [6, 0, 21, 44, 1, 61, 43, 1] the target: 61
when input is [58] the target: 52
when input is [58, 52] the target: 43
when input is [58, 52, 43] the target: 57
when input is [58, 52, 43, 57] the target: 57
when input is [58, 52, 43, 57, 57] the target: 

### Baseline: Bigram Language Model

In [155]:
from torch import nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.tocken_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx):
        return self.tocken_embedding_table(idx).transpose(1,2)

    def compute_loss(self, logits, targets):
        return F.cross_entropy(logits, targets)

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits = self(idx)
            last_time_step_logits = logits[:,:,-1]
            probs = F.softmax(last_time_step_logits, dim=1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx


m = BigramLanguageModel(vocab_size).to(device)
logits = m(xb)
loss = m.compute_loss(logits, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65, 8])
tensor(4.7863, device='cuda:0', grad_fn=<NllLoss2DBackward0>)


In [148]:
import numpy as np

-np.log(1/65)

4.174387269895637

In [156]:
initial_idx = torch.zeros((1,1), dtype=torch.long, device=device)
print(''.join(decode(m.generate(idx = initial_idx, max_new_tokens=100)[0].tolist())))


pYCXxfRkRZd
wc'wfNfT;OLlTEeC K
jxqPToTb?bXAUG:C-SGJO-33SM:C?YI3a
hs:LVXJFhXeNuwqhObxZ.tSVrddXlaSZaNe


In [160]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-2)

In [161]:
batch_size = 32
eval_iters = 200
for steps in range(10000):
    xb, yb = get_batch("train")
    logits = m(xb)
    loss = m.compute_loss(logits, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

2.3953356742858887


In [162]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(''.join(decode(m.generate(idx = context, max_new_tokens=500)[0].tolist())))


Th fepyotssththas l.
TAn.
Mourethal wave.
seazendee hee ovet aimous?




Cos cok ovet Hoie s inds he be fe f tas ny, ct Clo gscest hes,
A: du It n, moxcone.

NENobeakes aghercobun wsam k s withoumas Fond t s wllo INour id, mersed
Fourd?
TIOMyord po venond, d Cadity
K:
BIUSoou tiund thornofen e sutan wiporthare whanot, sitthers, spe Bllellke, on s h O, t pan, de wat d tive wout ir f; u;

Feknegineeee rtano iru forealoiroret HEnk;
CEdwhen tck is, d s, nd ftanofallon bay ho s, andesen, meseveminds 
